In [4]:
import numpy as np
data = np.load('/content/rho10000.npy', allow_pickle=True)
data = data[:,1:]

In [5]:
labels = data[:, 0].astype(int)
features = data[:, 1:].astype(np.float32)

In [6]:
def correlation_ratio(categories, measurements):
    categories = np.array(categories)
    measurements = np.array(measurements)
    cat_values = np.unique(categories)
    grand_mean = np.mean(measurements)

    ss_between = sum([
        len(measurements[categories == cat]) *
        (np.mean(measurements[categories == cat]) - grand_mean) ** 2
        for cat in cat_values
    ])

    ss_total = np.sum((measurements - grand_mean) ** 2)
    return ss_between / ss_total if ss_total != 0 else 0

for i, name in enumerate(['XxX', 'YxY', 'ZxZ']):
    eta_squared = correlation_ratio(labels, features[:, i])
    print(f"{name}: η² = {eta_squared:.3f}")


XxX: η² = 0.029
YxY: η² = 0.023
ZxZ: η² = 0.001


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier(random_state=42)
acc = cross_val_score(clf, features, labels, cv=5, scoring='accuracy')
print("Mean accuracy:", acc.mean())

Mean accuracy: 0.8


In [8]:
clf.fit(features, labels)
importances = clf.feature_importances_
for name, score in zip(['XxX', 'YxY', 'ZxZ'], importances):
    print(f'{name}: {score:.3f}')


XxX: 0.345
YxY: 0.351
ZxZ: 0.304


In [38]:
from sklearn.feature_selection import mutual_info_classif
mi = mutual_info_classif(features, labels)
print(dict(zip(['XxX', 'YxY', 'ZxZ'], mi)))


{'XxX': np.float64(0.7327456716263039), 'YxY': np.float64(0.7848375666975835), 'ZxZ': np.float64(0.6745042324401196)}


In [9]:
!pip install qiskit qiskit-machine-learning torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [10]:
# Normalize each feature to [0, 1]
min_vals = features.min(axis=0)
max_vals = features.max(axis=0)
features_scaled = (features - min_vals) / (max_vals - min_vals + 1e-8)

# Rescale to [-π, π]
features_scaled = features_scaled * (2 * np.pi) - np.pi

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
import torch

In [13]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    features_scaled, labels, test_size=0.2, stratify=labels, random_state=42
)

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [14]:
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import RealAmplitudes
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit.primitives import Estimator

In [33]:
import torch.nn as nn

In [32]:
from qiskit.quantum_info import Pauli

In [30]:
observables = [
    Pauli("ZII"), Pauli("IZI"), Pauli("IIZ"),  # Individual
    # Pauli("ZZI"), Pauli("ZIZ"), Pauli("IZZ")   # Pairwise
]

def create_qnn(num_qubits: int = 3):
    x = ParameterVector("x", num_qubits)
    circuit = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        circuit.ry(x[i], i)
    ansatz = RealAmplitudes(num_qubits=num_qubits, reps=2, entanglement="linear")
    circuit.compose(ansatz, inplace=True)

    return EstimatorQNN(
        circuit=circuit,
        input_params=x,
        weight_params=ansatz.parameters,
        estimator=Estimator(),
        input_gradients=True,
        observables=observables
    )

qnn = create_qnn()
model = TorchConnector(qnn)

# Full classifier
classifier = nn.Sequential(
    model,
    nn.Linear(3, 16, bias=True),
    nn.ReLU(),
    nn.Linear(16, 4, bias=True),
)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.LBFGS(classifier.parameters(), lr=1.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

for epoch in range(10):
    classifier.train()
    def closure():
        optimizer.zero_grad()
        inputs = X_train_tensor.float()
        if inputs.ndim == 1:
            inputs = inputs.unsqueeze(0)  # Ensure batch shape
        output = classifier(inputs)
        loss = criterion(output, y_train_tensor)
        loss.backward()
        return loss


    loss = optimizer.step(closure)
    scheduler.step()

    classifier.eval()
    with torch.no_grad():
        test_output = classifier(X_test_tensor)
        test_loss = criterion(test_output, y_test_tensor)
        preds = torch.argmax(test_output, dim=1)
        acc = (preds == y_test_tensor).float().mean().item()

    print(f"Epoch {epoch+1:02d}: Train Loss = {loss.item():.4f}, Test Acc = {acc:.4f}")

/tmp/ipython-input-30-2799938768.py:18: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator=Estimator(),
/tmp/ipython-input-30-2799938768.py:14: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  return EstimatorQNN(


Epoch 01: Train Loss = 1.4567, Test Acc = 0.9500
Epoch 02: Train Loss = 0.1910, Test Acc = 0.8000
Epoch 03: Train Loss = 0.0372, Test Acc = 0.8500
Epoch 04: Train Loss = 0.0003, Test Acc = 0.8500
Epoch 05: Train Loss = 0.0000, Test Acc = 0.8500
Epoch 06: Train Loss = 0.0000, Test Acc = 0.8500
Epoch 07: Train Loss = 0.0000, Test Acc = 0.8500
Epoch 08: Train Loss = 0.0000, Test Acc = 0.8500
Epoch 09: Train Loss = 0.0000, Test Acc = 0.8500
Epoch 10: Train Loss = 0.0000, Test Acc = 0.8500


In [31]:
torch.save(classifier.state_dict(), '/content/classifier_best_weights.pth')